<a href="https://colab.research.google.com/github/GIDISIA/RLDiplodatos/blob/master/stable_baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

Créditos:

* Documentación y repo de Stable-baselines https://stable-baselines3.readthedocs.io.
    * Tutorial sobre SB3: https://github.com/araffin/rl-tutorial-jnrr19.
* Documentación y repo de OpenAI Gym https://github.com/openai/gym/blob/master/docs/.
    * Crear un entorno https://github.com/openai/gym/blob/master/docs/creating-environments.md. 

Stable-baselines3: framework de deep RL que provee interfaces para ejecutar y adaptar algoritmos de RL "al estilo scikit-learn". Permite utilizar agentes abstrayéndonos de los detalles de bajo nivel de abstracción referentes a la implementación del algoritmo$^1$

Además, ofrece herramientas muy útiles como

* Monitores que permiten ver el rendimiento del agente según se desempeña en el entorno, sin tener que esperar a que finalice de entrenar.
* Callbacks que permiten accionar eventos cuando se cumplen algunas condiciones en el entrenamiento de nuestro agente (por ejemplo, detenerlo si la recompensa recibida es menor a cierto umbral tras un cierto período de tiempo).


Documentación https://stable-baselines3.readthedocs.io

Es un fork activamente mantenido de [OpenAI baselines](https://github.com/openai/baselines)

La versión 3 cambia el framework subyacente de Tensorflow a Pytorch y está activamente en desarrollo; no obstante la versión 2 es completamente funcional

$^1$ no obstante, al igual que sucede generalmente con librerías de ML: 

* Siempre es bueno tener en mente las características, ventajas y desventajas del algoritmo utilizado, pues de eso depende mucho la convergencia de nuestra solución, especialmente cuando se emplean entornos adaptados para nuestras necesidades. 

* Esta librería, al igual que demás frameworks generales de RL, están muy probadas en entornos estándares de RL como Atari o PyBullet. No obstante, es posible que nuestro entorno o nuestras necesidades difieran significativamente, lo que hace que en algunos casos haya que meter mano directo en el código de los algoritmos/librería.

# Interfaz básica stable-baselines

## Instalación

Desde Linux o Google Colab

In [1]:
!pip install stable-baselines3[extra,tests,docs]>=0.11.0a4 && pip install sb3-contrib

ERROR: sphinxcontrib-spelling 7.1.0 has requirement Sphinx>=3.0.0, but you'll have sphinx 1.8.5 which is incompatible.
  Using cached https://files.pythonhosted.org/packages/f6/62/796d8ae02732c162f8d53406f520c9f3c886a9ab24de4ef6995404c2b1d8/sphinxcontrib_spelling-7.1.0-py3-none-any.whl
ERROR: sphinxcontrib-spelling 7.1.0 has requirement Sphinx>=3.0.0, but you'll have sphinx 1.8.5 which is incompatible.


## Ejecución de un algoritmo de RL

### Importaciones/inicializaciones

In [2]:
import os
from subprocess import Popen, PIPE

import numpy as np
import matplotlib.pyplot as plt

import gym
from gym import spaces
#from gym.envs.registration import register

from stable_baselines3 import DQN, PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.env_util import make_vec_env

os.makedirs('logs', exist_ok=True)

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

cwd = os.getcwd()

%matplotlib inline
%load_ext tensorboard

### Ejemplo básico

In [3]:
env = gym.make('CartPole-v1')

# MlpPolicy es una política "estándar" que aprende con perceptron multicapa
# (es decir sin capas convolucionales o demás variantes),
# 2 capas ocultas con 64 neuronas cada una
model = DQN('MlpPolicy', env)
model.learn(total_timesteps=10000)


### Renderización

In [4]:
if not IN_COLAB:

    obs = env.reset()
    for i in range(1000):
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        env.render()
        if done:
          obs = env.reset()

    env.close()

### Logging

#### Ver rendimiento del agente en tensorboard

In [5]:
env = gym.make('CartPole-v1')

model = DQN('MlpPolicy', env, tensorboard_log='tensorboard/')
model.learn(total_timesteps=100000)

Para verlo en tensorboard, correr

`tensorboard --logdir=tensorboard/`

In [22]:
%tensorboard --logdir=tensorboard

<IPython.core.display.Javascript object>

### Monitor

Vamos a crear un monitor para loguear nuestro agente en la carpeta logs. Nuestro monitor guardará datos de recompensa (r), duración (l) y tiempo total (t)

In [7]:
env = gym.make('CartPole-v1')
env = Monitor(env, 'logs/')  # reemplazamos env por su monitor

model = DQN('MlpPolicy', env, )
model.learn(total_timesteps=10000)

### Callbacks

In [8]:
env = gym.make('CartPole-v1')

callbacks = []  # lista de callbacks a usar, pueden ser varios

# callback para detener entrenamiento al alcanzar recompensa de 9.8
# (a fines demostrativos, es una recompensa baja)
stop_training_callback = StopTrainingOnRewardThreshold(reward_threshold=9.8)

# al crear EvalCallback, se asocia el mismo con stop_training_callback
callbacks.append(EvalCallback(env, 
                              eval_freq=1000,
                              callback_on_new_best=stop_training_callback))

# la semilla aleatoria hace que las ejecuciones sean estocásticas
model = DQN('MlpPolicy', env, seed=42)
model.learn(total_timesteps=10000, callback=callbacks)

/usr/local/lib/python3.6/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Eval num_timesteps=1000, episode_reward=9.60 +/- 0.80
Episode length: 9.60 +/- 0.80
New best mean reward!
Eval num_timesteps=2000, episode_reward=9.20 +/- 0.75
Episode length: 9.20 +/- 0.75
Eval num_timesteps=3000, episode_reward=9.80 +/- 0.75
Episode length: 9.80 +/- 0.75
New best mean reward!


### Ejecutar agente RL en múltiples ambientes

Esta librería provee una interfaz para ejecutar agentes en varias instancias de un mismo entorno a la vez (*vectorized environments*), de modo tal que se habilite la ejecución paralela y de otras funcionalidades útiles.

Para ello, varios de sus algoritmos implementan cambios que consideren la posibilidad de que haya múltiples entornos subyacentes, por ejemplo `step(accion)` cambia a `step(lista_acciones)`, aplicando acciones a todos los entornos, recibiendo ahora múltiples observaciones y recompensas.

Otro cambio: se aplica `reset()` automáticamente a cada entorno que llega a un estado final.

SB brinda dos formas de utilizar entornos vectorizados:

* DummyVecEnv, el cuál consiste en un *wrapper* de varios entornos, los cuáles funcionarán en un sólo hilo. Este wrapper es útil como entrada de algoritmos que requieren los entornos de esta forma, y habilita los procesamientos y operaciones comunes de los entornos vectorizados (ejemplo: el *stacking* de 4 imágenes en entornos de tipo Atari).
* SubprocVecEnv, el cuál agrupa varios entornos que serán ejecutados en paralelo. Atención! **Puede comer mucha RAM**

Vemos un ejemplo:

In [9]:
# ejemplo de ambiente dummy
venv = DummyVecEnv([lambda: gym.make('CartPole-v1')]*4)

model = PPO('MlpPolicy', venv, )
model.learn(total_timesteps=10000)

También puede hacerse con un una función de SB a tal efecto

In [10]:
venv = make_vec_env(lambda: env, n_envs=4)

model = PPO('MlpPolicy', venv, )
model.learn(total_timesteps=10000)

### Ejecutar agente con políticas personalizadas

In [11]:
# Creamos una clase con una red neuronal de 128x128 neuronas

model = PPO('MlpPolicy', policy_kwargs=dict(net_arch=[128,128]), env='CartPole-v1', verbose=1).learn(total_timesteps=10000)
model.learn(total_timesteps=10000)

Using cpu device
Creating environment from the given name 'CartPole-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Utilizar un entorno personalizado

Antes que nada, además de la interfaz que ya vimos de Gym, hay otras nociones que tenemos que tener en cuenta en este contexto:

* Los entornos definen un espacio de estados y de acciones, a partir de los cuáles los modelos asumen y respetan la "forma" de observaciones y acciones. Por ejemplo, algunos algoritmos están diseñados para espacios de acciones discretos (DQN), continuos (DDPG) o bien poseen implementaciones particulares pueden usarse en ambos (PPO, en el repo de SB3). En cuanto a los espacios, algunos algoritmos asumen explícitamente un espacio discreto (y pequeño), como Q-Learning, mientras que otros como PPO asumen cualquier tipo de espacio.
* Los dos tipos más comunes de estados o acciones son los espacios discretos `gym.spaces.Discrete` y los continuos `gym.spaces.Box`.
* Los espacios discretos definen un conjunto de $n$ estados/acciones $\{ 0, 1, \dots, n-1 \}$, mientras que los espacios continuos definen un espacio $\mathbb{R}^d$, de una de las siguientes 4 formas: $[a, b], (-\infty, b], [a, \infty), (-\infty, \infty)$, en donde $a,b$ son las cotas superior e inferior (de existir).
* Ejemplos: un espacio de acciones `Discrete(4)` tiene 4 acciones: $\{0,1,2,3\}$; un espacio de estados `Discrete(16)` tiene 16 estados. Un espacio de estados ALTURA, ANCHO, N_CANALES que represente una imagen RGB acotada en $[a=0, b=255]$ se puede crear como

`observation_space = spaces.Box(low=0, high=255, shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)`

Para usar un entorno compatible por esta librería, el mismo tiene que heredar de *gym.Env*. Vemos un ejemplo (crédito: https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/5_custom_gym_env.ipynb)

**TODO translate below**

In [12]:
class GoLeftEnv(gym.Env):
  """
  Custom Environment that follows gym interface.
  This is a simple env where the agent must learn to go always left. 
  """
  # Because of google colab, we cannot implement the GUI ('human' render mode)
  metadata = {'render.modes': ['console']}
  # Define constants for clearer code
  LEFT = 0
  RIGHT = 1

  def __init__(self, grid_size=10):
    super(GoLeftEnv, self).__init__()

    # Size of the 1D-grid
    self.grid_size = grid_size
    # Initialize the agent at the right of the grid
    self.agent_pos = grid_size - 1

    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions, we have two: left and right
    n_actions = 2
    self.action_space = spaces.Discrete(n_actions)
    # The observation will be the coordinate of the agent
    # this can be described both by Discrete and Box space
    self.observation_space = spaces.Box(low=0, high=self.grid_size,
                                        shape=(1,), dtype=np.float32)

  def reset(self):
    """
    Important: the observation must be a numpy array
    :return: (np.array) 
    """
    # Initialize the agent at the right of the grid
    self.agent_pos = self.grid_size - 1
    # here we convert to float32 to make it more general (in case we want to use continuous actions)
    return np.array([self.agent_pos]).astype(np.float32)

  def step(self, action):
    if action == self.LEFT:
      self.agent_pos -= 1
    elif action == self.RIGHT:
      self.agent_pos += 1
    else:
      raise ValueError("Received invalid action={} which is not part of the action space".format(action))

    # Account for the boundaries of the grid
    self.agent_pos = np.clip(self.agent_pos, 0, self.grid_size)

    # Are we at the left of the grid?
    done = bool(self.agent_pos == 0)

    # Null reward everywhere except when reaching the goal (left of the grid)
    reward = 1 if self.agent_pos == 0 else 0

    # Optionally we can pass additional info, we are not using that for now
    info = {}

    return np.array([self.agent_pos]).astype(np.float32), reward, done, info

  def render(self, mode='console'):
    if mode != 'console':
      raise NotImplementedError()
    # agent is represented as a cross, rest as a dot
    print("." * self.agent_pos, end="")
    print("x", end="")
    print("." * (self.grid_size - self.agent_pos))

  def close(self):
    pass

In [13]:
env = GoLeftEnv(grid_size=10)
env = make_vec_env(lambda: env, n_envs=1)

model = PPO('MlpPolicy', env, verbose=1).learn(5000)

Using cpu device


Ejercicio: extender este entorno. Algunas ideas:

* Transformarlo en una grilla 2D, añadir paredes /trampas / agua.
* **TODO**

# RL-baselines zoo

Colección de agentes RL y herramientas útiles para ejecutarlos, evaluarlos e incluso hacer videos con ellos. Los agentes de este repo están preparados con la configuración requerida para los distintos tipos de entornos, incluyendo Atari, PyBullet y entornos clásicos, incluyendo configuraciones e híper-parámetros que producen buenas políticas para tales entornos.

Esta librería ofrece un muy buen punto de partida para utilizar agentes / entornos personalizados, ya que ofrece una [interfaz](https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/train.py) fácilmente adaptable a nuestras necesidades.

## Instalación

Desde Google Colab

In [14]:
if IN_COLAB:
    !git clone --recursive https://github.com/DLR-RM/rl-baselines3-zoo
    !cd rl-baselines3-zoo/
    !apt-get install swig cmake ffmpeg
    !pip install -r /content/rl-baselines3-zoo/requirements.txt

fatal: destination path 'rl-baselines3-zoo' already exists and is not an empty directory.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
  Using cached https://files.pythonhosted.org/packages/f6/62/796d8ae02732c162f8d53406f520c9f3c886a9ab24de4ef6995404c2b1d8/sphinxcontrib_spelling-7.1.0-py3-none-any.whl
ERROR: sphinxcontrib-spelling 7.1.0 has requirement Sphinx>=3.0.0, but you'll have sphinx 1.8.5 which is incompatible.


Desde Linux, ejecutando

    git clone --recursive https://github.com/DLR-RM/rl-baselines3-zoo
    cd rl-baselines3-zoo/
    sudo apt-get install swig cmake ffmpeg
    pip install -r requirements.txt

## Ejecución

Los agentes pueden ser llamados desde la consola mediante comandos como

`python train.py --algo algo_name --env env_id`

Los cuales pueden ser llamados usando

In [15]:
os.chdir('rl-baselines3-zoo/')

args = [
    '-n', str(100000),
    '--algo', 'ppo',
    '--env', 'CartPole-v1'
]

p = Popen(['python', 'train.py'] + args,
                               stdin=PIPE, stdout=PIPE, stderr=PIPE)
output, err = p.communicate()
rc = p.returncode
os.chdir(cwd)
assert rc == 0

Ver en acción el agente entrenado (nota: no disponible en Google Colab)

In [16]:
if not IN_COLAB:
    pass  # TODO mostrar el uso del enjoy

Grabar video! **TODO**

Ver curva de aprendizaje obtenida por el agente desde *utils.plot*

In [17]:
print(cwd)

/content


In [18]:
os.chdir(cwd)


In [19]:
os.chdir('rl-baselines3-zoo/')

args = [
    'ppo',
    'CartPole-v1',
    'logs/',
    'steps'
]

p = Popen(['python', '-m', 'scripts.plot_train'] + args, stdout=PIPE)
output, err = p.communicate()
rc = p.returncode
os.chdir(cwd)

assert rc == 0

In [20]:
print(output)

b'<Figure size 800x200 with 1 Axes>\n'


## Normalización de features y recompensas

**TODO**

## Híper-parámetros

RL-Baselines Zoo provee funcionalidad para optimizar los híper-parámetros con la librería [Optuna]( https://github.com/optuna/optuna). En los mismos se incluyen rangos de híper-parámetros que se usaron para optimizar entornos como los de PyBullet, y son fácilmente modificables para adaptarlo a nuestros propios entornos. Para ver cómo se llama a la interfaz de Optuna ver [este código](https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/utils/hyperparams_opt.py).

Nota: **Consume muchos recursos!**

**TODO**

# Recursos adicionales

* Framework adicional de aprendizaje por refuerzos a gran escala: https://docs.ray.io/en/master/rllib.html.
* Awesome Deep RL: https://github.com/kengz/awesome-deep-rl
* [Comunidad de Bots de RL para Rocket League](https://rlbot.org/)


* **TODO**

FIN